### Clean Time_dim 

In [3]:
import pandas as pd
time_dim_df = pd.read_csv("../data/time_dim.csv")
listing_new_df = pd.read_csv("../data/list_new_clean.csv")
print(time_dim_df.head())

         date  year  month  quarter day_of_week
0  2024-12-25  2024     12        4   Wednesday
1  2024-12-26  2024     12        4    Thursday
2  2024-12-27  2024     12        4      Friday
3  2024-12-28  2024     12        4    Saturday
4  2024-12-29  2024     12        4      Sunday


## Chunk 1: schema & Data types 

In [8]:

time_dim_df['date']= pd.to_datetime(time_dim_df['date'],errors='coerce')
time_dim_df['day_of_week']= time_dim_df['day_of_week'].astype(str)

In [ ]:
missing_dates = listing_new_df[~listing_new_df['date_listed'].isin(time_dim_df['date'])]
print("Missing dates count:", missing_dates['date_listed'].nunique())

### Find missing dates in listings that dont how up in our time_dim
missing_dates = listing_new_df[~listing_new_df['date_listed'].isin(time_dim_df['date'])]['date_listed'].unique()
print(missing_dates)




Missing dates count: 137
['2025-06-17' '2025-06-21' '2025-05-16' '2025-05-30' '2025-06-03'
 '2025-06-10' '2025-04-11' '2025-06-19' '2025-06-05' '2025-04-29'
 '2025-06-02' '2025-05-28' '2025-06-13' '2025-03-31' '2025-05-20'
 '2025-05-08' '2025-06-20' '2025-03-28' '2025-01-06' '2025-06-12'
 '2025-05-21' '2025-05-07' '2025-05-14' '2025-04-10' '2025-04-25'
 '2025-04-17' '2025-06-11' '2025-05-23' '2025-05-13' '2025-05-09'
 '2025-06-18' '2025-05-26' '2025-06-07' '2025-03-10' '2025-04-07'
 '2025-06-16' '2025-06-09' '2025-05-27' '2025-05-12' '2025-05-06'
 '2025-03-22' '2025-06-14' '2025-06-04' '2025-05-01' '2025-06-06'
 '2025-04-02' '2025-04-24' '2025-03-20' '2025-04-01' '2025-05-10'
 '2025-05-19' '2025-06-15' '2025-05-29' '2025-02-14' '2025-05-22'
 '2025-05-11' '2025-03-26' '2025-03-29' '2025-04-26' '2025-03-07'
 '2025-03-14' '2025-03-04' '2025-05-24' '2025-05-17' '2025-03-19'
 '2025-02-04' '2025-01-20' '2025-01-03' '2025-06-01' '2025-05-15'
 '2025-04-09' '2025-05-02' '2025-06-08' '2025-03-13

In [ ]:
# Step 1: Find missing unique dates relative to old time_dim_df
missing_dates = listing_new_df[~listing_new_df['date_listed'].isin(time_dim_df['date'])]['date_listed'].dropna().unique()
missing_dates = pd.to_datetime(missing_dates)
listing_new_df['date_listed']= pd.to_datetime(listing_new_df['date_listed'])





# Step 2: Create rows for missing dates
new_time_rows = pd.DataFrame({
    'date': missing_dates,
    'year': missing_dates.year,
    'month': missing_dates.month,
    'quarter': missing_dates.quarter,
    'day_of_week': missing_dates.day_name()
})

# Step 3: Append to existing time dimension
time_dim_updated = pd.concat([time_dim_df, new_time_rows], ignore_index=True)

# Step 4: Drop duplicate dates
time_dim_updated = time_dim_updated.drop_duplicates(subset=['date'])

# Step 5: Check missing dates again
still_missing = listing_new_df[~listing_new_df['date_listed'].isin(time_dim_updated['date'])]['date_listed'].dropna().unique()

print("Missing dates count after update:", len(still_missing))

Missing dates count after update: 0


In [28]:
## Cut off point 6 months 
cutoff_date = pd.to_datetime("2025-01-01")

# Filter listings_df to only keep rows where date_listed >= cutoff_date
listing_new_df = listing_new_df[listing_new_df['date_listed'] >= cutoff_date].copy()
time_dim_updated= time_dim_updated[time_dim_updated['date']>= cutoff_date].copy()

print(f"Remaining listings after filtering: {len(listing_new_df)}")
print(f"Remaining listings after filtering: {len(time_dim_updated)}")
still_missing = listing_new_df[~listing_new_df['date_listed'].isin(time_dim_updated['date'])]['date_listed'].dropna().unique()

print("Missing dates count after update:", len(still_missing))

Remaining listings after filtering: 3305
Remaining listings after filtering: 174
Missing dates count after update: 0


In [29]:
time_dim_updated.to_csv("time_dim_clean.csv",index=False)
listing_new_df.to_csv("list_new_clean.csv",index=False)

Edit